# 一、简化企业名称

In [1]:
import requests
import math
from concurrent.futures.thread import ThreadPoolExecutor
from collections import Counter
import pandas as pd

In [2]:
company_list = pd.read_excel('/Users/nanapower/Library/Containers/com.tencent.xinWeChat/Data/Library/Application Support/com.tencent.xinWeChat/2.0b4.0.9/0a5ee70a49025e211f9ccc484a9c8ff4/Message/MessageTemp/27ba9480ba2304984c4a1e9a4793d843/File/企业id获取结果.xlsx')
company_list

,全称,简称,secCode,orgId,Unnamed: 4
0,江苏天正电气有限公司,天正电气,605066,9900039721,605066
1,宏润建设集团股份有限公司,宏润建设,002062,9900000362,2062
2,江西三鑫医疗科技股份有限公司,三鑫医疗,300453,9900023828,300453
3,山东墨龙石油机械股份有限公司,山东墨龙,002490,gshk0000568,2490
4,安德利集团有限公司,安德利,603031,9900024937,603031
...,...,...,...,...,...
590,四川腾邦科技有限公司,*ST腾邦,300178,9900016528,300178
591,四川盛弘机械有限公司,盛弘股份,300693,9900032564,300693
592,江苏兴达新材料有限公司,兴达泡塑,834090,gfbj0834090,834090
593,山东赛特建材股份有限公司,赛特新材,688398,9900030638,688398


In [3]:
id = list(zip(range(1,len(company_list.全称)+1),company_list.全称,company_list.secCode,company_list.orgId))
id

[(1, '江苏天正电气有限公司', '605066', '9900039721'),
 (2, '宏润建设集团股份有限公司', '002062', '9900000362'),
 (3, '江西三鑫医疗科技股份有限公司', '300453', '9900023828'),
 (4, '山东墨龙石油机械股份有限公司', '002490', 'gshk0000568'),
 (5, '安德利集团有限公司', '603031', '9900024937'),
 (6, '华联机械集团有限公司', '000036', 'gssz0000036'),
 (7, '精华制药集团股份有限公司', '002349', '9900010389'),
 (8, '吉林省正和药业集团股份有限公司', '832257', 'gfbj0832257'),
 (9, '江苏腾龙生物药业有限公司', '603158', '9900023476'),
 (10, '广东旭龙物联科技股份有限公司', '430490', 'gfbj0430490'),
 (11, '福建华博教育科技股份有限公司', '831308', 'gfbj0831308'),
 (12, '安庆永强农业科技股份有限公司', '832903', 'gfbj0832903'),
 (13, '山东九州信泰信息科技股份有限公司', '838240', 'gfbj0838240'),
 (14, '无锡万吉科技股份有限公司', '838104', 'gfbj0838104'),
 (15, '奇华光电（昆山）股份有限公司', '836641', 'gfbj0836641'),
 (16, '山东易构软件技术股份有限公司', '834002', 'gfbj0834002'),
 (17, '河南康耀电子股份有限公司', '831524', 'gfbj0831524'),
 (18, '宁波亚茂光电股份有限公司', '831693', 'gfbj0831693'),
 (19, '济南天辰智能装备股份有限公司', '831234', 'gfbj0831234'),
 (20, '珠海美合科技股份有限公司', '834412', 'gfbj0834412'),
 (21, '淄博天恒纳米新材料科技股份有限公司', '873531', 'g

In [27]:
import time
# 功能二
def select(code, orgid):
    current = time.time()
    time.sleep(0.1)
    # 获取报告类型
    # category="category_ndbg_szsh;"

    # 获取时间
    start = '2011-01-01'
    end = '2023-04-21'

    # 根据股票代码的头文字，判断股票交易所信息
    if str(code)[0] == '6':
        column = 'sse'
        plate = 'sh'
    elif str(code)[0] == '8' or str(code)[0] == '4':
        column = 'third'
        plate = 'neeq'
    else:  # 3 0
        column = 'szse'
        plate = 'sz'

    # 设置初始页码
    page_num = 1
    # 设置初始列表存储筛选结果
    pdf_list = []
    while True:
        # 设置报告筛选参数
        
        params = {
            'stock': '{},{}'.format(code, orgid),
            'tabName': 'fulltext',
            'pageSize': '30',
            'pageNum': str(page_num),
            'category': '',#category,
            'seDate': '{}~{}'.format(start, end),
            'column': column,
            'plate': plate,
            'searchkey': '',
            'secid': '',
            'sortName': '',
            'sortType': '',
            'isHLtitle': 'true'
        }

       # 发起报告搜索请求
        try:
            r = requests.post('http://www.cninfo.com.cn/new/hisAnnouncement/query', params=params)
            r_json = r.json()
        except:
             return  [['无搜索结果！','无搜索结果！']]
        
        # 判断是否搜索失败、或者无搜索结果，如果无结果则结束
        if r_json['announcements'] == None or len(r_json['announcements']) == 0 :
            return  [['无搜索结果！','无搜索结果！']]
        # 遍历搜索结果
        for i in r_json['announcements']:
            pdf_list.append([i['announcementTitle'], 'http://static.cninfo.com.cn/'+i['adjunctUrl']])
        if math.ceil(r_json['totalRecordNum']/30) ==page_num:
            break
        page_num += 1
    return pdf_list

In [5]:
def run(index,name,code, orgid):
    lis = select(code, orgid)
    if lis == [['无搜索结果！','无搜索结果！']]:
        with open(f'/Users/nanapower/学习资料/python_learnning/模块与项目/AI/图灵AI/巨潮资讯/年报爬取/pdf未处理/!{index}_{name}.txt','w') as f:
            for i in lis:
                f.write(i[0]+' ')
                f.write(i[1]+'\n')
            print(index,'下载失败！')
            return
    else:
        with open(f'/Users/nanapower/学习资料/python_learnning/模块与项目/AI/图灵AI/巨潮资讯/年报爬取/pdf未处理/{index}_{name}.txt','w') as f:
            for i in lis:
                f.write(i[0]+' ')
                f.write(i[1]+'\n')
            print(index,'下载完成！')

In [6]:
id

[(1, '江苏天正电气有限公司', '605066', '9900039721'),
 (2, '宏润建设集团股份有限公司', '002062', '9900000362'),
 (3, '江西三鑫医疗科技股份有限公司', '300453', '9900023828'),
 (4, '山东墨龙石油机械股份有限公司', '002490', 'gshk0000568'),
 (5, '安德利集团有限公司', '603031', '9900024937'),
 (6, '华联机械集团有限公司', '000036', 'gssz0000036'),
 (7, '精华制药集团股份有限公司', '002349', '9900010389'),
 (8, '吉林省正和药业集团股份有限公司', '832257', 'gfbj0832257'),
 (9, '江苏腾龙生物药业有限公司', '603158', '9900023476'),
 (10, '广东旭龙物联科技股份有限公司', '430490', 'gfbj0430490'),
 (11, '福建华博教育科技股份有限公司', '831308', 'gfbj0831308'),
 (12, '安庆永强农业科技股份有限公司', '832903', 'gfbj0832903'),
 (13, '山东九州信泰信息科技股份有限公司', '838240', 'gfbj0838240'),
 (14, '无锡万吉科技股份有限公司', '838104', 'gfbj0838104'),
 (15, '奇华光电（昆山）股份有限公司', '836641', 'gfbj0836641'),
 (16, '山东易构软件技术股份有限公司', '834002', 'gfbj0834002'),
 (17, '河南康耀电子股份有限公司', '831524', 'gfbj0831524'),
 (18, '宁波亚茂光电股份有限公司', '831693', 'gfbj0831693'),
 (19, '济南天辰智能装备股份有限公司', '831234', 'gfbj0831234'),
 (20, '珠海美合科技股份有限公司', '834412', 'gfbj0834412'),
 (21, '淄博天恒纳米新材料科技股份有限公司', '873531', 'g

In [27]:
run(2, '宏润建设集团股份有限公司', '002062', '9900000362')

1
37
2
37
3
37
4
37
5
37
6
37
7
37
8
37
9
37
10
37
11
37
12
37
13
37
14
37
15
37
16
37
17
37
18
37
19
37
20
37
21
37
22
37
23
37
24
37
25
37
26
37
27
37
28
37
29
37
30
37
31
37
32
37
33
37
34
37
35
37
36
37
37
37
38
37
[['宏润建设-2022年年度股东大会见证法律意见书', 'http://static.cninfo.com.cn/finalpage/2023-04-19/1216455278.PDF'], ['2022年度股东大会决议公告', 'http://static.cninfo.com.cn/finalpage/2023-04-19/1216455277.PDF'], ['关于召开2022年度股东大会的提示性公告', 'http://static.cninfo.com.cn/finalpage/2023-04-11/1216367815.PDF'], ['2022年社会责任报告', 'http://static.cninfo.com.cn/finalpage/2023-03-29/1216246362.PDF'], ['2022年度总经理工作报告', 'http://static.cninfo.com.cn/finalpage/2023-03-29/1216246361.PDF'], ['2022年年度报告', 'http://static.cninfo.com.cn/finalpage/2023-03-29/1216246360.PDF'], ['2022年度内部控制规则落实自查表', 'http://static.cninfo.com.cn/finalpage/2023-03-29/1216246359.PDF'], ['2022年年度审计报告', 'http://static.cninfo.com.cn/finalpage/2023-03-29/1216246357.PDF'], ['关于2023年度日常关联交易额度预计的公告', 'http://static.cninfo.com.cn/finalpage/2023-03-29/12

In [7]:
import os

In [8]:
lis = os.listdir('/Users/nanapower/学习资料/python_learnning/模块与项目/AI/图灵AI/巨潮资讯/年报爬取/pdf未处理/')
lis = [i.split('_')[0] for i in lis]
for i in id:
    if str(i[0]) in lis:
            print(str(i[0]),'已下载！')
            continue
    run(i[0],i[1],i[2],i[3])

1 已下载！
2 已下载！
3 已下载！
4 下载完成！
5 已下载！
6 已下载！
7 下载完成！
8 已下载！
9 下载完成！
10 下载完成！
11 下载完成！
12 下载完成！
13 下载完成！
14 下载完成！
15 下载完成！
16 下载完成！
17 下载完成！
18 下载完成！
19 下载完成！
20 下载完成！
21 下载完成！
22 下载完成！
23 下载完成！
24 下载失败！
25 下载完成！
26 下载完成！
27 下载完成！
28 下载完成！
29 下载完成！
30 下载完成！
31 下载完成！
32 下载完成！
33 下载完成！
34 下载完成！
35 下载完成！
36 下载完成！
37 下载完成！
38 下载失败！
39 下载完成！
40 下载完成！
41 下载完成！
42 下载完成！
43 下载完成！
44 下载完成！
45 下载完成！
46 下载完成！
47 下载完成！
48 下载完成！
49 下载完成！
50 下载完成！
51 下载完成！
52 下载完成！
53 下载完成！
54 下载完成！
55 下载完成！
56 下载完成！
57 下载完成！
58 下载完成！
59 下载完成！
60 下载完成！
61 下载完成！
62 下载完成！
63 下载完成！
64 下载完成！
65 下载完成！
66 下载完成！
67 下载完成！
68 下载完成！
69 下载完成！
70 下载完成！
71 下载完成！
72 下载完成！
73 下载完成！
74 下载完成！
75 下载完成！
76 下载完成！
77 下载完成！
78 下载完成！
79 下载完成！
80 下载完成！
81 下载完成！
82 下载完成！
83 下载完成！
84 下载完成！
85 下载完成！
86 下载完成！
87 下载完成！
88 下载完成！
89 下载完成！
90 下载完成！
91 下载完成！
92 下载完成！
93 下载完成！
94 下载完成！
95 下载完成！
96 下载完成！
97 下载完成！
98 下载完成！
99 下载完成！
100 下载完成！
101 下载失败！
102 下载完成！
103 下载完成！
104 下载失败！
105 下载完成！
106 下载完成！
107 下载完成！
108 下载完成！
109 下载完成！
110 下载完成！
111 下载完成！
112 

In [29]:
lis = os.listdir('/Users/nanapower/学习资料/python_learnning/模块与项目/AI/图灵AI/巨潮资讯/年报爬取/pdf未处理/')
lis = [i.split('_')[0] for i in lis]
lis = [i for i in lis if i.startswith('!')]
# lis = [i[1:] for i in lis]
lis

['!316', '!373', '!371', '!337']

In [30]:
lis = os.listdir('/Users/nanapower/学习资料/python_learnning/模块与项目/AI/图灵AI/巨潮资讯/年报爬取/pdf未处理/')
lis = [i.split('_')[0] for i in lis]
lis = [i for i in lis if i.startswith('!')]
lis = [i[1:] for i in lis]
lis

['316', '373', '371', '337']

In [ ]:
run(104, '河北彩客新材料科技股份有限公司', '01986', '9900030799')

In [31]:
for i in id:
    if str(i[0]) not in lis:
        # print(str(i[0]),'已下载！')
        continue
    run(i[0],i[1],i[2],i[3])
    print(i)

316 下载失败！
(316, '深圳市海普瑞药业集团股份有限公司', '149870', '9900012288')
337 下载失败！
(337, '深圳市特发信息股份有限公司', '149666', 'gssz0000070')
371 下载失败！
(371, '广西柳工机械股份有限公司', '149532', 'gssz0000528')
373 下载失败！
(373, '南京磐能电力科技股份有限公司', 'T00016', '9900008368')


In [28]:
run(267, '芯原微电子（上海）股份有限公司', '688521','9900040648')

267 下载完成！
